# Day 4

In [1]:
import $ivy.`org.scalaz::scalaz-core:7.2.22`
import $ivy.`org.scalaz::scalaz-effect:7.2.22`
import $ivy.`org.scalaz::scalaz-typelevel:7.1.17`
import $ivy.`org.scalaz::scalaz-scalacheck-binding:7.2.21`
import scalaz._
import Scalaz._
import scalacheck.ScalazProperties._
import scalacheck.ScalazArbitrary._
import scalacheck.ScalaCheckBinding._

import $ivy.$                               

import $ivy.$                                 

import $ivy.$                                    

import $ivy.$                                             

import scalaz._

import Scalaz._

import scalacheck.ScalazProperties._

import scalacheck.ScalazArbitrary._

import scalacheck.ScalaCheckBinding._

## Functor則

In [2]:
List(1, 2, 3) map {identity} assert_=== List(1, 2, 3)

In [3]:
(List(1, 2, 3) map {{(_: Int) * 3} map {(_: Int) + 1}}) assert_=== (List(1, 2, 3) map {(_: Int) * 3} map {(_: Int) + 1})

In [4]:
functor.laws[List].check

+ functor.invariantFunctor.identity: OK, passed 100 tests.
+ functor.invariantFunctor.composite: OK, passed 100 tests.
+ functor.identity: OK, passed 100 tests.
+ functor.composite: OK, passed 100 tests.


## 法則を破る

In [5]:
sealed trait COption[+A] {}
case class CSome[A](counter: Int, a: A) extends COption[A]
case object CNone extends COption[Nothing]

implicit def coptionEqual[A]: Equal[COption[A]] = Equal.equalA
implicit val coptionFunctor = new Functor[COption] {
    def map[A, B](fa: COption[A])(f: A => B): COption[B] = fa match {
        case CNone => CNone
        case CSome(c, a) => CSome(c + 1, f(a))
    }
}

defined trait COption
defined class CSome
defined object CNone
defined function coptionEqual
coptionFunctor: AnyRef with Functor[COption] = $sess.cmd4Wrapper$Helper$$anon$1@d874d5c

In [6]:
(CSome(0, "ho"): COption[String]) map {(_: String) + "ha"}

res5: COption[String] = CSome(1,hoha)

In [7]:
(CSome(0, "ho"): COption[String]) map {identity}

res6: COption[String] = CSome(1,ho)

In [7]:
functor.laws[COption].check()

cmd7.sc:1: could not find implicit value for parameter af: org.scalacheck.Arbitrary[cmd7Wrapper.this.cmd4.wrapper.COption[Int]]
val res7 = functor.laws[COption].check()
                       ^

: 

In [8]:
import org.scalacheck.Gen
import org.scalacheck.Arbitrary

import org.scalacheck.Gen

import org.scalacheck.Arbitrary

In [9]:
implicit def COptionArbiterary[A](implicit a: Arbitrary[A]): Arbitrary[COption[A]] =
    a map { a => (CSome(0, a): COption[A]) }

defined function COptionArbiterary

In [10]:
functor.laws[COption].check

! functor.invariantFunctor.identity: Falsified after 0 passed tests.
> ARG_0: CSome(0,-1)
! functor.invariantFunctor.composite: Falsified after 0 passed tests.
> ARG_0: CSome(0,1)
> ARG_1: org.scalacheck.ArbitraryArities$$Lambda$3016/1183814913@aa955f0
> ARG_2: org.scalacheck.ArbitraryArities$$Lambda$3016/1183814913@1eeaac3f
> ARG_3: org.scalacheck.ArbitraryArities$$Lambda$3016/1183814913@44baac20
> ARG_4: org.scalacheck.ArbitraryArities$$Lambda$3016/1183814913@3ccc17e3
! functor.identity: Falsified after 0 passed tests.
> ARG_0: CSome(0,1569514195)
! functor.composite: Falsified after 0 passed tests.
> ARG_0: CSome(0,2147483647)
> ARG_1: org.scalacheck.ArbitraryArities$$Lambda$3016/1183814913@37ca03b5
> ARG_2: org.scalacheck.ArbitraryArities$$Lambda$3016/1183814913@268072c0


## Applicative則

```scala
trait ApplicativeLaw extends FunctorLaw {
    def identityAp[A](fa: F[A])(implicit FA: Equal[F[A]]): Boolean =
        FA.equal(ap(fa)(point((a: A) => a)), fa)

    def composition[A, B, C](fbc: F[B => C], fab: F[A => B], fa: F[A])(implicit FC: Equal[F[C]]) =
        FC.equal(ap(ap(fa)(fab))(fbc), ap(fa)(ap(fab)(ap(fbc)(point((bc: B => C) => (ab: A => B) => bc compose ab)))))

    def homomorphism[A, B](ab: A => B, a: A)(implicit FB: Equal[F[B]]): Boolean =
        FB.equal(ap(point(a))(point(ab)), point(ab(a)))

    def interchange[A, B](f: F[A => B], a: A)(implicit FB: Equal[F[B]]): Boolean =
        FB.equal(ap(point(a))(f), ap(f)(point((f: A => B) => f(a))))
}
```

## Semigroup則

```scala
trait SemigroupLaw {
    def associative(f1: F, f2: F, f3: F)(implicit F: Equal[F]): Boolean =
        F.equal(append(f1, append(f2, f3)), append(append(f1, f2), f3))
}
```

In [11]:
semigroup.laws[Int @@ Tags.Multiplication].check

+ semigroup.associative: OK, passed 100 tests.


## Monoid則

```scala
  /**
   * Monoid instances must satisfy [[scalaz.Semigroup.SemigroupLaw]] and 2 additional laws:
   *
   *  - '''left identity''': `forall a. append(zero, a) == a`
   *  - '''right identity''' : `forall a. append(a, zero) == a`
   */
  trait MonoidLaw extends SemigroupLaw {
    def leftIdentity(a: F)(implicit F: Equal[F]) = F.equal(a, append(zero, a))
    def rightIdentity(a: F)(implicit F: Equal[F]) = F.equal(a, append(a, zero))
  }
```

In [12]:
1 * 2 assert_=== 2

In [13]:
2 * 1 assert_=== 2

In [13]:
(Monoid[Int @@ Tags.Multiplication].zero |+| Tags.Multiplication(2): Int) assert_=== 2

cmd13.sc:1: type mismatch;
 found   : Int @@ scalaz.Tags.Multiplication
    (which expands to)  AnyRef{type Tag = scalaz.Tags.Multiplication; type Self = Int}
 required: Int
val res13 = (Monoid[Int @@ Tags.Multiplication].zero |+| Tags.Multiplication(2): Int) assert_=== 2
                                                     ^

: 

In [13]:
(Tags.Multiplication(2) |+| Monoid[Int @@ Tags.Multiplication].zero: Int) assert_=== 2

cmd13.sc:1: type mismatch;
 found   : Int @@ scalaz.Tags.Multiplication
    (which expands to)  AnyRef{type Tag = scalaz.Tags.Multiplication; type Self = Int}
 required: Int
val res13 = (Tags.Multiplication(2) |+| Monoid[Int @@ Tags.Multiplication].zero: Int) assert_=== 2
                                    ^

: 

In [14]:
monoid.laws[(Int @@ Tags.Multiplication)].check

+ monoid.semigroup.associative: OK, passed 100 tests.
+ monoid.left identity: OK, passed 100 tests.
+ monoid.right identity: OK, passed 100 tests.


## Monoid としての Option

```scala
  implicit def optionMonoid[A: Semigroup]: Monoid[Option[A]] = new Monoid[Option[A]] {
    def append(f1: Option[A], f2: => Option[A]) = (f1, f2) match {
      case (Some(a1), Some(a2)) => Some(Semigroup[A].append(a1, a2))
      case (Some(a1), None)     => f1
      case (None, Some(a2))     => f2
      case (None, None)         => None
    }

    def zero: Option[A] = None
  }
```

In [15]:
(none: Option[String]) |+| "andy".some

res14: Option[String] = Some("andy")

In [16]:
(Ordering.LT: Ordering).some |+| none

res15: Option[Ordering] = Some(LT)

In [17]:
Tags.First('a'.some) |+| Tags.First('b'.some)

res16: Option[Char] @@ Tags.First = Some(a)

In [18]:
Tags.First(none: Option[Char]) |+| Tags.First('b'.some)

res17: Option[Char] @@ Tags.First = Some(b)

In [19]:
Tags.First('a'.some) |+| Tags.First(none: Option[Char])

res18: Option[Char] @@ Tags.First = Some(a)

In [20]:
Tags.Last('a'.some) |+| Tags.Last('b'.some)

res19: Option[Char] @@ Tags.Last = Some(b)

In [21]:
Tags.Last(none: Option[Char]) |+| Tags.Last('b'.some)

res20: Option[Char] @@ Tags.Last = Some(b)

In [22]:
Tags.Last('a'.some) |+| Tags.Last(none: Option[Char])

res21: Option[Char] @@ Tags.Last = Some(a)

## Foldable

```scala
trait Foldable[F[_]] { self =>
  /** Map each element of the structure to a [[scalaz.Monoid]], and combine the results. */
  def foldMap[A,B](fa: F[A])(f: A => B)(implicit F: Monoid[B]): B

  /**Right-associative fold of a structure. */
  def foldRight[A, B](fa: F[A], z: => B)(f: (A, => B) => B): B

  ...
}
```

In [23]:
List(1, 2, 3).foldRight (1) {_ * _}

res22: Int = 6

In [24]:
9.some.foldLeft(2) {_ + _}

res23: Int = 11

In [25]:
List(1, 2, 3) foldMap {identity}

res24: Int = 6

In [26]:
List(true, false, true, true) foldMap {Tags.Disjunction.apply}

res25: Boolean @@ Tags.Disjunction = true